<a href="https://colab.research.google.com/github/juanjosemarinm/SyS2025-2/blob/main/prueba_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.7 MB/s eta 0:00:00


In [2]:
!mkdir pages

In [3]:
%%writefile 0_👋_Hello.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import os
from scipy.fft import rfft, rfftfreq

st.set_page_config(
    page_title="Dashboard AM y Circuitos",
    page_icon="👋",
)

st.title("🔊 Dashboard Interactivo")

menu = st.sidebar.radio("Selecciona una sección", ["📻 Modulación AM", "🔌 Circuitos Eléctricos"])

if menu == "📻 Modulación AM":
    st.header("🎧 Modulación AM de audio YouTube")

    link = st.text_input("URL de YouTube", "https://www.youtube.com/watch?v=nmFdyBUAsfQ")
    ti = st.slider("Tiempo inicial [s]", 0, 50, 20)
    tf = st.slider("Tiempo final [s]", ti+1, 60, 25)
    fc = st.slider("Frecuencia portadora [Hz]", 1000, 100000, 60000, step=1000)

    if st.button("Procesar audio"):
        os.system(f"yt-dlp --extract-audio -o 'audio' --audio-format mp3 {link}")
        os.system("ffmpeg -y -i audio.mp3 output.wav")

        x, fs = sf.read("output.wav")
        xpro = x.copy()
        xs = xpro[int(ti*fs):int(tf*fs), :]
        tt = np.arange(ti, tf, 1/fs)

        st.audio("output.wav")

        Im = 1
        Ac = float(np.max(np.abs(xs))/Im)
        C = Ac * np.cos(2*np.pi*fc*tt)
        C_reshaped = C.reshape(-1,1)
        y = (1 + xs/Ac) * C_reshaped
        d = y * C_reshaped

        XfC = np.fft.fft(C)
        Xfxs = np.fft.fft(xs[:,0])
        Xfy = np.fft.fft(y[:,0])
        Xfd = np.fft.fft(d[:,0])
        vfre = np.fft.fftfreq(len(C), 1/fs)

        fig, ax = plt.subplots()
        ax.plot(vfre, abs(Xfy), label='Modulada')
        ax.plot(vfre, abs(Xfxs), label='Mensaje')
        ax.plot(vfre, abs(XfC), label='Portadora')
        ax.plot(vfre, abs(Xfd), label='Demodulada')
        ax.set_xlim(0, fs/2)
        ax.legend(); ax.set_title("Espectros")
        st.pyplot(fig)

elif menu == "🔌 Circuitos Eléctricos":
    st.header("⚡ Simulación de circuito con carga resistiva y RC")

    fs = 10000
    t = np.linspace(0, 0.1, int(fs*0.1), endpoint=False)
    f = st.slider("Frecuencia de la fuente (Hz)", 10, 1000, 50)
    vin = 10 * np.sin(2 * np.pi * f * t)
    vout = np.abs(vin)

    R = st.number_input("Resistencia (Ohm)", value=100)
    iR = vout / R

    C = st.number_input("Capacitancia (F)", value=470e-6, format="%.1e")
    dt = 1/fs
    vC = np.zeros_like(t)
    iRC = np.zeros_like(t)

    for n in range(1, len(t)):
        dv = (vout[n-1] - vC[n-1]) / R
        iRC[n] = dv
        vC[n] = vC[n-1] + dt * (iRC[n]/C)

    def compute_thd(signal, fs, f_base):
        yf = np.abs(rfft(signal))
        xf = rfftfreq(len(signal), 1/fs)
        f_res = fs / len(signal)
        i_fund = np.argmax((xf >= f_base - f_res) & (xf <= f_base + f_res))
        V1 = yf[i_fund]
        Vn = np.delete(yf[:len(yf)//2], i_fund)
        thd = np.sqrt(np.sum(Vn**2)) / V1
        return thd

    thd_R = compute_thd(iR, fs, f)
    thd_RC = compute_thd(iRC, fs, f)
    fpd_R = 1 / np.sqrt(1 + thd_R**2)
    fpd_RC = 1 / np.sqrt(1 + thd_RC**2)

    st.markdown(f"**THD (Resistiva):** {thd_R:.2%} — **FPD:** {fpd_R:.2%}")
    st.markdown(f"**THD (RC):** {thd_RC:.2%} — **FPD:** {fpd_RC:.2%}")

    fig, axs = plt.subplots(2, 1, figsize=(10, 6))
    axs[0].plot(t, iR, label=f"Resistiva - THD = {thd_R:.2%}")
    axs[0].plot(t, iRC, label=f"RC - THD = {thd_RC:.2%}")
    axs[0].legend(); axs[0].grid(); axs[0].set_title("Corrientes")

    axs[1].plot(t, vin, label="Fuente")
    axs[1].plot(t, vC, label="Voltaje en C")
    axs[1].legend(); axs[1].grid(); axs[1].set_title("Voltajes")

    st.pyplot(fig)

Writing 0_👋_Hello.py


In [4]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Hello.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-10 00:11:23--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-10 00:11:24--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-10T00%3A48%3A05Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-1

In [5]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")


Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
